# 使用ChArUco Board做相機校正


## 先取得校正要用到的點

In [1]:
import cv2
import cv2.aruco as aruco

cap = cv2.VideoCapture('CharUco_board.mp4') # 取得攝影機
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()  # 建立Aruco參數(detectMarker process會用到)
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX  #用subpixel的方式來收細角點

arucoDict    = aruco.Dictionary_get(aruco.DICT_6X6_250)  #我們用的是這個參數產生的ChArUco board

# 必須描述ChArUco board的尺寸規格
gridX        = 5 # 水平方向5格
gridY        = 7 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
# ArUco marker為2cmX2cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)  # 建立ChArUco board

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH))) # 取得攝影機的長寬
refinedStrategy = True  #
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
frameId        = 0
collectCorners = []
collectIds     = []
collectFrames  = []
while True:
    ret, frame = cap.read()  # 讀取一幀影像
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))  # 縮小畫面
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)  #偵測Aruco marker的角點位置(每個marker的四個角點), ids是每個marker的id, rejected是有錯或信心不足的marker的corner(也許是不符呵我們設定的DICT_6X6_250)

    #corners是17個 4*2的陣列(代表每個marker的四個角點)
    #ids是17個 1*1的陣列(代表每個marker的id) 每個marker的id是唯一的
    #rejected是多個 4*2的陣列(代表每個rejected的四個角點)

    #refinedStrategy代表是否用板子資訊去強化arucro的偵測
    #這個一定要在detectMarker之後再做
    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)

    #每100個frame存一次之後校正要用到的點對應關係
    if frameId % 100 == 50 and len(ids)==17: # 17 ArUco markers 如果全部正確被讀到才起來
        collectCorners.append(corners)
        collectIds.append(ids.ravel())
        collectFrames.append(frame)

    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)  #把marker的角點畫出來

    cv2.imshow('find points for calibration',frame)
    if cv2.waitKey(20) != -1:
        break

    frameId += 1

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0


## 開始校正

In [2]:
import numpy as np
caliCorners=np.concatenate([np.array(x).reshape(-1,4,2) for x in collectCorners],axis=0)
counter=np.array([len(x) for x in collectIds])
caliIds=np.array(collectIds).ravel()
cameraMatrixInit = np.array([[ 1000.,    0., frameWidth/2.],[    0., 1000., frameHeight/2.],[    0.,    0.,           1.]])
distCoeffsInit   = np.zeros((5,1))
ret, aruco_cameraMatrix, aruco_distCoeffs, aruco_rvects, aruco_tvects = aruco.calibrateCameraAruco(caliCorners,caliIds,counter,charucoBoard,(frameWidth,frameHeight),cameraMatrixInit,distCoeffsInit)
print(aruco_cameraMatrix)
print(aruco_distCoeffs)

[[913.81070779   0.         479.5586767 ]
 [  0.         921.0914589  293.00945991]
 [  0.           0.           1.        ]]
[[ 0.06739156]
 [-0.11176159]
 [-0.00446193]
 [-0.00266387]
 [-0.01974274]]


In [3]:
caliCorners=[]
caliIds    =[]
for corners, ids, frame in zip(collectCorners,collectIds,collectFrames):
    ret, charucoCorners, charucoIds = aruco.interpolateCornersCharuco(corners,ids,frame,charucoBoard,aruco_cameraMatrix,aruco_distCoeffs)
    caliCorners.append(charucoCorners)
    caliIds.append(charucoIds)

ret, charuco_cameraMatrix, charuco_distCoeffs, charuco_rvects, charuco_tvects = aruco.calibrateCameraCharuco(caliCorners,caliIds,charucoBoard,(frameWidth,frameHeight), aruco_cameraMatrix,aruco_distCoeffs)
print(charuco_cameraMatrix)
print(charuco_distCoeffs)

[[888.25457479   0.         482.34262359]
 [  0.         896.25270996 305.57029901]
 [  0.           0.           1.        ]]
[[ 0.08518861]
 [-0.41241448]
 [-0.00235481]
 [-0.00228344]
 [ 1.0810133 ]]


# 用校正好的相機找到每個CharUco_board的位置(3D位置)

In [4]:
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
cap = cv2.VideoCapture('CharUco_board.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if ids is not None and len(ids)>0:
        aruco.drawDetectedMarkers(frame, corners, ids)
        ret, rvect, tvect = aruco.estimatePoseBoard(corners, ids, charucoBoard, charuco_cameraMatrix, charuco_distCoeffs, None, None)
        if ret:
            aruco.drawAxis(frame, charuco_cameraMatrix, charuco_distCoeffs, rvect, tvect, squareSize)

    cv2.imshow('Estimation of the pose of a CharUco board with intrinsic camera parameters',frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()

# 用校正好的相機找到每個Aruco的位置

In [6]:
markerSize  = 6 #6cm
cap = cv2.VideoCapture('ArUco_marker.mp4')
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

hw2_output_video = cv2.VideoWriter('myHW2Video.mp4', cv2.VideoWriter_fourcc(*'XVID'),
                                   cap.get(cv2.CAP_PROP_FPS),
                                   (frameWidth,frameHeight))  #輸出的影片

nggyu_cap = cv2.VideoCapture('nggyu.mp4')
hyyyy_cap = cv2.VideoCapture('hyyyy.mp4')
esg_cap   = cv2.VideoCapture('esg.mp4')
ncat_cap = cv2.VideoCapture('ncat.mp4')
game_cap = cv2.VideoCapture('game.mp4')
vibe_cap = cv2.VideoCapture('vibe.mp4')

has_started = [False for _ in range(6)] #紀錄哪些id開始被偵測到了
while True:
    ret, frame = cap.read()
    _, nggyu_frame = nggyu_cap.read()
    _, hyyyy_frame = hyyyy_cap.read()
    _, esg_frame = esg_cap.read()
    _, ncat_frame = ncat_cap.read()
    _, game_frame = game_cap.read()
    _, vibe_frame = vibe_cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))

    # #把nggyu影片貼合到一張新影像的左上角
    # nggyu_frame_full = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]),dtype=np.uint8)
    # nggyu_frame_full[:360,:480]=nggyu_frame[:360,90:90+480]
    #
    # #把hyyyy影片貼合到一張新影像的左上角
    # hyyyy_frame_full = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]),dtype=np.uint8)
    # hyyyy_frame_full[:360,:480]=hyyyy_frame[:360,:480]

    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if len(corners) > 0:
        #aruco.drawDetectedMarkers(frame, corners, ids)
        rvects, tvects, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, charuco_cameraMatrix, charuco_distCoeffs)
        for corner,id in zip(corners,ids):
            if id == 1:
                mask_image = np.ones_like(vibe_frame)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,0.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(vibe_frame, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 2:
                mask_image = np.ones_like(esg_frame)
                source_pts = np.array([[640.0,480.0],[0.0,480.0],[0.0,00.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(esg_frame, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 3:
                mask_image = np.ones_like(nggyu_frame)
                source_pts = np.array([[470.0+90,360.0],[90.0,360.0],[90.0,0.0],[470.0+90,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(nggyu_frame, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 4:
                mask_image = np.ones_like(hyyyy_frame)
                source_pts = np.array([[480.0,360.0],[0.0,360.0],[0.0,0.0],[480.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(hyyyy_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 5:
                mask_image = np.ones_like(ncat_frame)
                source_pts = np.array([[640.0,480.0],[80.0,480.0],[80.0,0.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(ncat_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 6:
                mask_image = np.ones_like(game_frame)
                source_pts = np.array([[754.0,480.0],[100.0,480.0],[100.0,0.0],[754.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(game_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]

    #aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('Estimation of the pose of arUco marker with intrinsic camera parameters',frame)
    hw2_output_video.write(frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()
hw2_output_video.release()
nggyu_cap.release()
hyyyy_cap.release()
esg_cap.release()
ncat_cap.release()
game_cap.release()
vibe_cap.release()
